In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# from paths import mkl_data
import requests # API library

import random
import time
import os

from isoweek import Week

import requests # API library

import numpy as np
import pandas as pd
import json
import re

import torch
import pyro
import pyro.distributions as dist
from pyro.contrib.autoguide import AutoDiagonalNormal, AutoMultivariateNormal
from pyro.infer import MCMC, NUTS, HMC, SVI, Trace_ELBO
from pyro.optim import Adam, ClippedAdam
from pyro.infer import Predictive



In [14]:
# df_m1 = pd.read_csv('../df_matches_clean.csv')
df_m1 = pd.read_csv('../data/model1_X_train.csv')
df_m1_test = pd.read_csv('../data/model1_X_test.csv')  

unique_coaches = pd.concat([df_m1['team1_coach_id'], df_m1['team2_coach_id']]).unique()
#Because of model change from (-1,0,1) -> (0,1,2)
df_m1['team1_win'] = df_m1['team1_win'] + 1
df_m1_test['team1_win'] = df_m1_test['team1_win'] + 1

In [24]:
def skill_model(coach_results):
    skills = {}
    
    # Setting up hyperpriors for coach mu
    mu_hyper_mu = pyro.sample("mu_hyper_mu", dist.Normal(0, 10))
    mu_hyper_sigma = pyro.sample("mu_hyper_sigma", dist.LogNormal(0, 1))
    
    # Setting up hyperpriors for coach sigma
    sigma_hyper_scale = pyro.sample("sigma_hyper_scale", dist.HalfCauchy(1))

    with pyro.plate("coaches_plate", len(unique_coaches)):
        for i, coach_id in enumerate(unique_coaches):            
            # Each coach's mu has a normal prior
            mu = pyro.sample(f"mu_{coach_id}", dist.Normal(mu_hyper_mu, mu_hyper_sigma))
            # Each coach's sigma has a HalfCauchy prior
            sigma = pyro.sample(f"sigma_{coach_id}", dist.HalfCauchy(sigma_hyper_scale))
            skills[coach_id] = pyro.sample(f"skill_{coach_id}", dist.Normal(mu, sigma))
    
    with pyro.plate("games_plate", len(coach_results)):
        team1_coaches = coach_results['team1_coach_id'].values
        team2_coaches = coach_results['team2_coach_id'].values
        results = coach_results['team1_win'].values
        
        for i in range(len(coach_results)):
            skill_diff = skills[team1_coaches[i]] - skills[team2_coaches[i]]
            logits = torch.tensor([skill_diff, 0.0, -skill_diff])
            outcome_probs = torch.softmax(logits, 0)
            pyro.sample(f"result_{i}", dist.Categorical(outcome_probs), obs=torch.tensor(results[i]))



In [16]:
def guide(coach_results):
    for coach_id in unique_coaches:
        mu_loc = pyro.param(f"{coach_id}_mu_loc", torch.tensor(0.0))
        mu_scale = pyro.param(f"{coach_id}_mu_scale", torch.tensor(1.0), constraint=dist.constraints.positive)
        sigma_loc = pyro.param(f"{coach_id}_sigma_loc", torch.tensor(1.0))
        sigma_scale = pyro.param(f"{coach_id}_sigma_scale", torch.tensor(0.05), constraint=dist.constraints.positive)
        
        mu = pyro.sample(f"mu_{coach_id}", dist.Normal(mu_loc, mu_scale))
        sigma = pyro.sample(f"sigma_{coach_id}", dist.LogNormal(sigma_loc, sigma_scale))


In [25]:
optimizer = Adam({"lr": 0.01})
svi = SVI(skill_model, guide, optimizer, loss=Trace_ELBO())

# Training loop
num_steps = 5000
# X_torch = torch.tensor(df_m1['team1_win'].values, dtype=torch.int32)
for step in range(num_steps):
    loss = svi.step(df_m1)
    if step % 100 == 0:
        print(f"Step {step} : loss = {loss}")

# Fetch updated parameter estimates
updated_params = {key: pyro.param(key).item() for key in pyro.get_param_store().keys()}
# print(updated_params)


ValueError: only one element tensors can be converted to Python scalars
         Trace Shapes:        
          Param Sites:        
         Sample Sites:        
      mu_hyper_mu dist       |
                 value       |
   mu_hyper_sigma dist       |
                 value       |
sigma_hyper_scale dist       |
                 value       |
    coaches_plate dist       |
                 value  3611 |
         mu_18963 dist  3611 |
                 value       |
      sigma_18963 dist  3611 |
                 value       |
      skill_18963 dist  3611 |
                 value  3611 |
          mu_2525 dist  3611 |
                 value       |
       sigma_2525 dist  3611 |
                 value       |
       skill_2525 dist  3611 |
                 value  3611 |
        mu_223328 dist  3611 |
                 value       |
     sigma_223328 dist  3611 |
                 value       |
     skill_223328 dist  3611 |
                 value  3611 |
        mu_131680 dist  3611 